In [103]:
# workspace()
module Acrobots

using FixedSizeArrays
using Quaternions
import Base: convert

type State{PositionType, VelocityType}
    position::PositionType
    velocity::VelocityType
end

abstract Position{N, T} <: FixedVectorNoTuple{N, T}

immutable AcrobotPosition{T} <: Position{2, T}
    theta1::T
    theta2::T
    function AcrobotPosition(a::NTuple{2, T})
        new{T}(a[1], a[2])
    end
end

abstract Velocity{N, T} <: FixedVectorNoTuple{N, T}

immutable AcrobotVelocity{T} <: Velocity{2, T}
    theta1::T
    theta2::T
    function AcrobotVelocity(a::NTuple{2, T})
        new{T}(a[1], a[2])
    end
end

abstract Input{N, T} <: FixedVectorNoTuple{N, T}

immutable AcrobotInput{T} <: Input{1, T}
    tau::T
    function AcrobotInput(a::NTuple{1, T})
        new{T}(a[1])
    end
end

typealias AcrobotState{T} State{AcrobotPosition{T}, AcrobotVelocity{T}}

abstract Manipulator

type AcrobotLink{T}
    length::T
    mass::T
    damping::T
    length_to_CoM::T
    inertia::T
end

type Acrobot{T} <: Manipulator
    links::NTuple{2, AcrobotLink{T}}
    gravity::T
end
    
function dynamics{State, Input}(robot::Manipulator, time, state::State, input::Input)
    H, C, B = manipulator_dynamics(robot, state)
    H_inv = inv(H)
    tau = B * destructure([input]) - C
    vdot = H_inv * tau
    State(state.velocity, vec(vdot))
end

function manipulator_dynamics{T}(robot::Acrobot, state::AcrobotState{T})
    inertias_about_joint = [robot.links[i].inertia + robot.links[i].mass * robot.links[i].length_to_CoM^2 for i in 1:2]
    m2l1lc2 = robot.links[2].mass * robot.links[1].length * robot.links[2].length_to_CoM
    
    c = cos(state.position)
    s = sin(state.position)
    s12 = sin(state.position.theta1 + state.position.theta2)
    
    h12 = inertias_about_joint[2] + m2l1lc2 * c[2]
    H = [inertias_about_joint[1] + inertias_about_joint[2] + robot.links[2].mass * robot.links[1].length^2 + 2 * m2l1lc2 * c[2]   h12;
        h12                                                                                                                       inertias_about_joint[2]]
    C = [-2 * m2l1lc2 * s[2] * state.velocity.theta2     -m2l1lc2 * s[2] * state.velocity.theta2;
         m2l1lc2 * s[2] * state.velocity.theta1          0]
    G = robot.gravity * [robot.links[1].mass * robot.links[1].length_to_CoM * s[1] + robot.links[2].mass * (robot.links[1].length * s[1] + robot.links[2].length_to_CoM * s12);
                         robot.links[2].mass * robot.links[2].length_to_CoM * s12]
#     v = destructure([state.velocity])
    Cv = Float64[dot(AcrobotVelocity{T}(vec(C[i,:])), state.velocity) + G[i] + robot.links[i].damping * state.velocity[i] for i = 1:2]
#     Cv = Array(Float64, 2)
#     for i = 1:2
#         Cv[i] = G[i] + robot.links[i].damping * state.velocity[i]
#         for j = 1:2
#             Cv[i] += C[i,j] * state.velocity[j]
#         end
#     end
        
    B = Float64[0; 1]
    
    H, Cv, B
end
    

acrobot() = Acrobot((AcrobotLink(1.0, 1.0, 0.1, 0.5, 0.083), AcrobotLink(2.0, 1.0, 0.1, 1.0, 0.33)), 9.81)


convert{PosType <: Position, VelType <: Velocity}(::Type{PosType}, state::State{PosType, VelType}) = state.position
convert{PosType <: Position, VelType <: Velocity}(::Type{VelType}, state::State{PosType, VelType}) = state.velocity

using PyLCM
using PyCall

@pyimport drake as lcmdrake

function viewer_data(link::Acrobots.AcrobotLink, name)
    msg = lcmdrake.lcmt_viewer_link_data()
    msg[:name] = name
    msg[:robot_num] = 1
    msg[:num_geom] = 1
    
    geom = lcmdrake.lcmt_viewer_geometry_data()
    geom[:type] = geom[:BOX]
    geom[:position] = [0; 0; -link.length / 2]
    quat = qrotation([0.; 1; 0], pi/2)
    geom[:quaternion] = [quat.s; quat.v1; quat.v2; quat.v3]
    geom[:color] = [0.2; 0.2; 0.8; 0.6]
    geom[:string_data] = ""
    geom[:float_data] = [link.length; link.length/10; link.length/10]
    geom[:num_float_data] = 3
    push!(msg["geom"], geom)
    
    msg
end

function viewer_data(robot::Acrobot)
    msg = lcmdrake.lcmt_viewer_load_robot()
    msg[:num_links] = 2
    for i in 1:2
        push!(msg["link"], viewer_data(robot.links[i], "link$(i)"))
    end
    msg
end

type DrakeVisualizer
    robot::Manipulator
    lcm::PyLCM.PyLCMWrapper
end

DrakeVisualizer(robot::Manipulator) = DrakeVisualizer(robot, LCM())

function load_robot(vis::DrakeVisualizer)
    load_msg = viewer_data(vis.robot)
    publish(vis.lcm, "DRAKE_VIEWER_LOAD_ROBOT", load_msg)
end


rotmat(theta) = [cos(theta) -sin(theta); sin(theta) cos(theta)]

function draw{T}(vis::DrakeVisualizer, state::AcrobotState{T})
    msg = lcmdrake.lcmt_viewer_draw()
    msg[:num_links] = 2
    msg[:link_name] = ["link1"; "link2"]
    msg[:robot_num] = [1; 1]
    
    p0 = [0; 0]
    p1 = rotmat(state.position.theta1) * [0; -vis.robot.links[1].length]
#     p2 = p1 + rotmat(state.position.theta1 + state.position.theta2) * [0; -robot.links[2].length]
    
    msg[:position] = Any[[0; 0; 0], [p1[1]; 0; p1[2]]]
    quats = [qrotation([0; -1; 0], state.position.theta1);
            qrotation([0.; -1; 0], state.position.theta1 + state.position.theta2)]
    msg[:quaternion] = Any[[q.s; q.v1; q.v2; q.v3] for q in quats]
    publish(vis.lcm, "DRAKE_VIEWER_DRAW", msg)
end
    

end

import Acrobots

In [104]:
robot = Acrobots.acrobot()
state = Acrobots.AcrobotState{Float64}([0;0], [0;0])

Acrobots.State{Acrobots.AcrobotPosition{Float64},Acrobots.AcrobotVelocity{Float64}}(Acrobots.AcrobotPosition{Float64}(0.0,0.0),Acrobots.AcrobotVelocity{Float64}(0.0,0.0))

In [105]:
vis = Acrobots.DrakeVisualizer(robot)
Acrobots.load_robot(vis)
state = Acrobots.AcrobotState{Float64}([pi/4;pi/4], [0;0])
Acrobots.draw(vis, state)

In [106]:
Acrobots.dynamics(robot, 0, Acrobots.AcrobotState{Float64}([1.; 2.], [3.; 4.]), Acrobots.AcrobotInput{Float64}(5.))

Acrobots.State{Acrobots.AcrobotPosition{Float64},Acrobots.AcrobotVelocity{Float64}}(Acrobots.AcrobotPosition{Float64}(3.0,4.0),Acrobots.AcrobotVelocity{Float64}(21.382653016334,-18.427570843977477))

In [114]:
using ProfileView

state = Acrobots.AcrobotState{Float64}(randn(2), zeros(2))
robot = Acrobots.acrobot()
input = Acrobots.AcrobotInput{Float64}(0)
dt = 0.001
# pygui(true)
# PyPlot.ion()
# fig = figure(1)
# clf()
# ax = fig[:add_subplot](1,1,1)
# vis = construct_visualizer(robot, ax)
vis = Acrobots.DrakeVisualizer(robot)
Acrobots.load_robot(vis)
Profile.clear()
ts = 0:dt:5
for t = ts
    statedot = Acrobots.dynamics(robot, t, state, input)
    state.position += statedot.position * dt
    state.velocity += statedot.velocity * dt
    Acrobots.draw(vis, state)
#     draw(vis, state)
#     PyPlot.draw()
end
# elapsed = @elapsed @profile for t = ts
#     statedot = Acrobots.dynamics(robot, t, state, input)
#     state.position += statedot.position * dt
#     state.velocity += statedot.velocity * dt
#     Acrobots.draw(vis, state)
# #     draw(vis, state)
# #     PyPlot.draw()
# end
# @show elapsed / length(ts)
# ProfileView.view()

In [32]:
using PyPlot

rotmat(theta) = [cos(theta) -sin(theta); sin(theta) cos(theta)]


type AcrobotVisualizer
    robot::Acrobot
    lines
end

function construct_visualizer(robot::Acrobot, ax)
    AcrobotVisualizer(robot, (ax[:plot]([0,0],[0,0], "bo-"), ax[:plot]([0,0], [0,0], "ro-")))
end

function draw(vis::AcrobotVisualizer, position::AcrobotPosition)
    p0 = [0; 0]
    p1 = rotmat(position.theta1) * [0; -robot.links[1].length]
    p2 = p1 + rotmat(position.theta1 + position.theta2) * [0; -robot.links[2].length]
    vis.lines[1][1][:set_xdata]([p0[1], p1[1]])
    vis.lines[1][1][:set_ydata]([p0[2], p1[2]])
    vis.lines[2][1][:set_xdata]([p1[1], p2[1]])
    vis.lines[2][1][:set_ydata]([p1[2], p2[2]])
#     plot(, , "bo-")
#     plot([p1[1], p2[1]], [p1[2], p2[2]], "ro-")
    axis("equal")
    xlim([-3, 3])
    ylim([-4, 1])
end

draw{Position, Velocity}(robot, state::State{Position, Velocity}) = draw(robot, convert(Position, state))

LoadError: LoadError: TypeError: AcrobotVisualizer: in type definition, expected Type{T}, got Module
while loading In[32], in expression starting on line 6

In [49]:
@which dot(state.velocity, state.velocity)

dot{T<:FixedSizeArrays.FixedArray{T,NDim,SIZE}}(a::T<:FixedSizeArrays.FixedArray{T,NDim,SIZE}, b::T<:FixedSizeArrays.FixedArray{T,NDim,SIZE}) at /Users/rdeits/.julia/v0.4/FixedSizeArrays/src/ops.jl:118